# AutoGen Pagrindinis Pavyzdys

Šiame kodo pavyzdyje naudosite [AutoGen](https://aka.ms/ai-agents/autogen) AI sistemą, kad sukurtumėte pagrindinį agentą.

Šio pavyzdžio tikslas yra parodyti jums veiksmus, kuriuos vėliau naudosime kituose kodo pavyzdžiuose, įgyvendindami skirtingus agentinius modelius.


## Importuokite reikalingas Python paketus


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Sukurkite klientą

Šiame pavyzdyje naudosime [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) norėdami pasiekti LLM.

`model` yra apibrėžtas kaip `gpt-4o-mini`. Pabandykite pakeisti modelį į kitą, kuris yra prieinamas GitHub Models rinkoje, kad pamatytumėte skirtingus rezultatus.

Kaip greitą testą, tiesiog paleisime paprastą užklausą - `Kokia yra Prancūzijos sostinė`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Agento apibrėžimas

Dabar, kai sukūrėme `client` ir patvirtinome, kad jis veikia, sukurkime `AssistantAgent`. Kiekvienam agentui galima priskirti:  
**name** - Trumpas pavadinimas, kuris bus naudingas naudojant jį kelių agentų srautuose.  
**model_client** - Klientas, kurį sukūrėte ankstesniame žingsnyje.  
**tools** - Galimi įrankiai, kuriuos agentas gali naudoti užduočiai atlikti.  
**system_message** - Metapranešimas, kuris apibrėžia užduotį, elgesį ir LLM toną.  

Galite pakeisti sistemos pranešimą, kad pamatytumėte, kaip LLM reaguoja. Apie `tools` kalbėsime 4-oje pamokoje.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Paleiskite agentą

Žemiau pateikta funkcija paleis agentą. Naudojame metodą `on_message`, kad atnaujintume agento būseną nauja žinute.

Šiuo atveju, būseną atnaujiname nauja vartotojo žinute, kuri yra `"Suplanuok man puikias saulėtas atostogas"`.

Galite pakeisti žinutės turinį, kad pamatytumėte, kaip LLM reaguoja skirtingai.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Atsakomybės atsisakymas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Dėl svarbios informacijos rekomenduojama profesionali žmogaus vertimo paslauga. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius naudojant šį vertimą.
